# Modeling Hurricane Data

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
# read in the storm landfall data
data = np.genfromtxt(os.path.join('data','hurricane_data','UShur1851-2010.txt'),
                     skip_header=1, delimiter='   ').astype(int)

# read in the storm data
dftrop = pd.read_csv(os.path.join('data','hurricane_data','tropical.txt'),sep='\t')
dftrop = dftrop.set_index('Year',drop=False)

## Plot the hurricane data

In [ ]:
plt.figure(figsize=(15,6))

# the timeseries
plt.subplot(1,2,1)
plt.bar(data[:,0], data[:,1])
plt.grid(linestyle='--',linewidth=0.5,alpha=0.4)
plt.ylabel('Number of Hurricanes')

# the distribution
plt.subplot(1,2,2)
plt.hist(data[:,1],bins=np.arange(-0.5,8.5),ec='k')
plt.grid(linestyle='--',linewidth=0.5,alpha=0.4)
plt.ylabel('Number of Years')
plt.xlabel('Number of Hurricanes That Made Landfall');

## Investigate Potential Poisson distributions

In [ ]:
def plot_poisson_distributions(lamb,df):
    
    k = np.arange(0,30)
    
    plt.hist(df['NamedStorms'], bins=np.arange(-0.5,30.5), ec='k', density=True)
    
    pmf_poisson = stats.poisson.pmf(k,lamb)
    plt.plot(k,pmf_poisson, 'k-')
    
    plt.grid(linestyle='--',linewidth=0.5,alpha=0.4)
    plt.ylabel('Number of Years')
    plt.xlabel('Number of Hurricanes')
    plt.gca().set_ylim([0,0.175])
    
    plt.title('$\\lambda = $' + '{:.2f}'.format(lamb))

In [ ]:
interact(plot_poisson_distributions,
         lamb=widgets.FloatSlider(min=0, max=30, step=0.2, value=5),
        df=fixed(dftrop));

## Poisson Distribution with a Constant Expected Value
We can estimate the expected value $\lambda$ of the of the Poisson distribution 
$$P(k|\lambda) = \frac{\lambda^k e^{-\lambda}}{k!}$$
using the Generalized Linear Model function of the statsmodel module. In particular, the following code will solve for $\beta_0$ such that
$$\beta_0 = \text{log}(u)$$

In [ ]:
result = smf.glm(formula='NamedStorms ~ 1',
                 data=dftrop,
                 family=sm.families.Poisson()).fit()
result.summary()
beta_0 = result.params.iloc[0]
Lambda = np.exp(beta_0)
print(Lambda)

## Climate Variables
In the next section, we consider the extent to which $\lambda$ might be related to climate variables such as the Southern Oscillation Index, the North Atlantic Oscillation, or the temperature in the North Atlantic. In other words, how can climate as predictor variables for the expected value $\lambda$ such that
$$\text{log}(\lambda_i) = \beta_0 + x_{1,i}\beta_1 + x_{2,i}\beta_2 + x_{3,i}\beta_3$$
First, we'll implement a function to read in the variables:

In [ ]:
def read_psl_file(psl_file):
    '''
    Read a data file in in NOAA Physical Sciences Laboratory (PSL) format
    
    Input: String containing the path to a PSL data file
    Returns: Pandas dataframe with monthly data in columns and the year as the index
    
    For a description of the PSL format see: https://psl.noaa.gov/gcos_wgsp/Timeseries/standard/
    '''
    
    f = open(psl_file, "r")
    all_lines = f.readlines()
    start_year = all_lines[0].split()[0]
    end_year = all_lines[0].split()[1]

    for i in range(1,len(all_lines)):
        stri = all_lines[i].find(end_year)
        if stri>=0:
            end_index = i

    missing_val = float(all_lines[end_index+1])
    nrows = int(end_year)-int(start_year)+1
    df = pd.read_csv(psl_file,skiprows=1,nrows=nrows,sep='\\s+',header=None,na_values=missing_val)
    df = df.rename(columns={0:'year'})
    df = df.set_index('year',drop=True)
    
    return df

Next, we read the variables in and add them to the dataframe:

In [ ]:
# read in the climate indices
dfsoi = read_psl_file(os.path.join('data','hurricane_data','soi.data'))
dftna = read_psl_file(os.path.join('data','hurricane_data','tna.data'))
dfnao = read_psl_file(os.path.join('data','hurricane_data','nao.data'))

dftrop['SOI'] = dfsoi.loc[:,5:6].mean(axis=1)
dftrop['TNA'] = dftna.loc[:,5:6].mean(axis=1)
dftrop['NAO'] = dfnao.loc[:,5:6].mean(axis=1)

df = dftrop.dropna()

And then we plot them to investigate:

In [ ]:
plt.figure(figsize=(10,7))

plt.subplot(3,1,1)
plt.bar(df['Year'], df['SOI'])
plt.title('Southern Oscillation Index')
plt.grid(linestyle='--', alpha=0.4, linewidth = 0.4)
plt.gca().set_xticklabels([])

plt.subplot(3,1,2)
plt.bar(df['Year'], df['TNA'])
plt.title('North Atlantic Temperature')
plt.grid(linestyle='--', alpha=0.4, linewidth = 0.4)
plt.gca().set_xticklabels([])

plt.subplot(3,1,3)
plt.bar(df['Year'], df['NAO'])
plt.title('North Atlantic Oscillation')
plt.grid(linestyle='--', alpha=0.4, linewidth = 0.4)


Using the same code as above, we investigate how the indices can be used as a predictor of the expected value in each year: 

In [ ]:
# Model: A constant and three climate indices
result = smf.glm(formula='NamedStorms ~ 1 + SOI + TNA + NAO',
                 data=df,
                 family=sm.families.Poisson()).fit()
print(result.summary())
beta = result.params
Lambda = np.exp(beta.iloc[0] + beta.iloc[1]*df['SOI'] +
                beta.iloc[2]*df['TNA'] + beta.iloc[3]*df['NAO'])


In [ ]:
plt.plot(df['Year'], Lambda, label='Model')
plt.plot(df['Year'], df['NamedStorms'], 'k.',label='data')
plt.title("Named Storms")
plt.legend()
plt.ylabel("Counts")
plt.grid(linestyle='--',linewidth=0.5,alpha=0.6)